# Índice

- [1. Conexión a la API de Yelp](#Conexion-a-la-API-de-YELP)
- [2. Análisis exploratorio de datos](#Exploracion-del-DataFrame)
- [3. Limpieza de datos](#Limpieza-de-datos)
- [4. Conclusiones](#Conclusiones-generales)

# Avance #2 del Proyecto Integrador 

### - Conectarse a la API de Yelp para obtener información de negocios locales de una ciudad según el país seleccionado.
### - Extraer datos como nombre del negocio, categoría, calificación, número de reseñas, calificación, etc.
### - Integrar esta información al análisis que se está realizando para una ciudad/país en específico
### - Documentar resultados
--------------------------

# Conociendo al Cliente 360°: Datos, Opiniones y Tendencias. | **Avance 2**

### Objetivo Principal
En este **segundo avance del proyecto integrador**, nos enfocamos en obtener y analizar nuevos datos sobre restaurantes de una ciudad específica para enriquecer nuestro análisis inicial.

### Ciudad Seleccionada: **Chicago**
Tras el análisis realizado en la primera etapa del proyecto, hemos decidido centrar nuestro estudio en la ciudad de **Chicago**

Este documento abarca un proceso integral que incluye:

1. **Conexión a la API de Yelp** - Configuración y obtención de datos
2. **Almacenamiento de información** - Recolección y estructuración de datos
3. **Proceso de limpieza** - Normalización y tratamiento de datos
4. **Análisis exploratorio** - Exploración inicial y detección de patrones

### Propósito Estratégico
El propósito central es **identificar insights relevantes** que complementen y enriquezcan los hallazgos obtenidos en la primera parte del proyecto, proporcionando una visión más completa y detallada del ecosistema gastronómico de Chicago.

---

### Preparación del entorno: Carga de librerías

Al igual que en la primera etapa del proyecto, es fundamental importar todas las librerías necesarias para asegurar un análisis eficiente y riguroso de los datos. Estas herramientas serán clave para la manipulación, limpieza, transformación y exploración de la información obtenida, así como para la detección y tratamiento de posibles anomalías en el dataset.

A continuación, se cargan las principales librerías de análisis de datos, procesamiento, conexión a APIs y manejo de cadenas, que permitirán desarrollar cada fase del proyecto de manera óptima.

In [1]:
# Carga de librerías necesarias

import pandas as pd # Manipulación de datos
import numpy as np # Operaciones matemáticas y estadísticas 
from IPython.display import display # Para mostrar DataFrames en Jupyter Notebook, aunque no es estrictamente necesario, es una buena práctica para visualizar los datos de manera más clara sin usar extensiones adicionales
from mega_funcion import imputar # Importamos la función imputar desde el archivo mega_funcion.py, que contiene la lógica para imputar valores en un DataFrame según diferentes criterios y operaciones
from fuzzywuzzy import fuzz # Librería para comparación difusa de cadenas, útil para detectar similitudes entre textos que pueden tener pequeñas variaciones o errores tipográficos
import requests # Para hacer solicitudes HTTP a la API de Yelp 
import ast # Para evaluar cadenas que representan estructuras de datos de Python (como listas o diccionarios) y convertirlas en objetos de Python reales


c:\Users\mfyz1\AppData\Local\Programs\Python\Python313\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## **Conexion a la API de YELP**

### Una vez cargadas las librerías necesarias, el siguiente paso es conectarse a la API de Yelp para obtener los datos de restaurantes en la ciudad de Chicago.

#### Nota: Por las restricciones de la API, solo es posible consultar hasta 200 registros de restaurantes utilizando únicamente el parámetro de ciudad. Por esta razón, el análisis se realiza con esta cantidad de datos. Si se requiere obtener un mayor volumen de información, el código correspondiente se encuentra disponible en un archivo adicional dentro de la carpeta del proyecto.

----
# Índice

- [1. Conexión a la API de Yelp](#Conexion-a-la-API-de-YELP)
- [2. Análisis exploratorio de datos](#Exploracion-del-DataFrame)
- [3. Limpieza de datos](#Limpieza-de-datos)
- [4. Conclusiones](#Conclusiones-generales)
--------

#### Variables de configuración para la API de Yelp

Antes de realizar la consulta, es necesario definir las variables de configuración que permitirán autenticar y estructurar correctamente la solicitud a la API de Yelp:

In [ ]:
# Variables de configuración para la API de Yelp

url = "https://api.yelp.com/v3/businesses/search" # URL desde donde se obtendrán los datos 
cliente_id = # ID del cliente para autenticación en la API de Yelp, reemplaza por tu propio ID
api_key =  # Clave de API para autenticación en la API de Yelp, reemplaza por tu propia clave

In [3]:
# Variables adicionales para la solicitud

headers = {'Authorization':'Bearer %s'%api_key} # Encabezados para la solicitud HTTP, incluyendo la autorización con la clave de API
ciudad = 'Chicago' # Ciudad para la cual se desean obtener los datos de restaurantes
params = {'term':'restaurants', 'location':ciudad, 'limit':50} # Parámetros para la solicitud, buscando restaurantes en la ciudad especificada, con un límite de 50 resultados por solicitud

A continuación, se realiza la solicitud GET a la API de Yelp utilizando los parámetros y credenciales definidos previamente. Esta consulta permite obtener hasta 50 resultados por petición, por lo que será necesario realizar varias solicitudes para alcanzar el límite máximo permitido por la API.

El siguiente código muestra cómo ejecutar la solicitud y visualizar la respuesta para verificar que la conexión se haya realizado correctamente (un código de estado 200 indica éxito).

In [4]:
# Realizar la solicitud GET a la API de Yelp

response = requests.get(url=url, params=params, headers=headers) # Realizar la solicitud GET a la API de Yelp con la URL, parámetros y encabezados especificados, esto solo nos traerá 50 resultados
response # Mostrar la respuesta de la solicitud para verificar que se haya realizado correctamente (código 200 indica éxito)

<Response [200]>

#### Ciclo para obtener hasta 200 resultados de la API de Yelp

Debido a las restricciones de la API de Yelp, solo es posible obtener hasta 200 registros utilizando el parámetro de ciudad. Para lograr esto, se realiza un ciclo que envía varias solicitudes (con el parámetro `offset`) para recolectar los datos en bloques de 50 resultados por petición, hasta alcanzar el límite permitido.

De esta manera, se asegura la obtención de la mayor cantidad de información posible para el análisis, respetando las limitaciones de la API.

In [5]:
# Procesar y guardar los datos obtenidos

all_results = [] # Lista para almacenar todos los resultados obtenidos de la API

for offset in range(0, 1000, 50):  # Iterar con un desplazamiento (offset) de 50 en 50 hasta 1000 
    
    params = {'term': 'restaurants', 'location': ciudad, 'limit': 50, 'offset': offset} # Actualizar los parámetros para incluir el offset
    response = requests.get(url, params=params, headers=headers) # Realizar la solicitud GET con los nuevos parámetros
    data = response.json() # Convertir la respuesta a formato JSON
    all_results.extend(data.get('businesses', [])) # Agregar los negocios obtenidos a la lista de todos los resultados, que en este caso sera la información de los restaurantes que se guardarán en un solo archivo .csv 
    
    if len(data.get('businesses', [])) < 50: # Si se obtienen menos de 50 resultados,
        break  # Parar el ciclo, ya que no hay más datos disponibles

df = pd.json_normalize(all_results) # Normalizar los datos JSON para convertirlos en un DataFrame de pandas 
df.to_csv('chicago_restaurants.csv', index=False) # Guardar el DataFrame en un archivo CSV sin incluir el índice, esto para no ejecutar nuevamente la API y tener los datos guardados localmente 

## **Exploracion del DataFrame**

### Ya que pudimos obtener los resultados de los requests de la API y los convertimos a un archivo .csv, procedemos a iniciar con la exploración inicial para poder encontrar los principales problemas a resolver

En este punto, contamos con un archivo `.csv` que contiene la información de los restaurantes obtenida desde la API de Yelp. El siguiente paso es realizar una exploración inicial de los datos para identificar posibles inconsistencias, valores nulos, columnas redundantes o cualquier otro problema que pueda afectar el análisis posterior. Esta etapa es fundamental para asegurar la calidad y utilidad de los datos antes de proceder con la limpieza y transformación.

----
# Índice

- [1. Conexión a la API de Yelp](#Conexion-a-la-API-de-YELP)
- [2. Análisis exploratorio de datos](#Exploracion-del-DataFrame)
- [3. Limpieza de datos](#Limpieza-de-datos)
- [4. Conclusiones](#Conclusiones-generales)
--------

In [6]:
df_original = pd.read_csv('chicago_restaurants.csv') # Cargamos el DataFrame desde el archivo .csv guardado previamente
df_res = df_original.copy() # Creamos una copia del DataFrame original para trabajar con él sin modificar el original 

### Conocemos la forma principal del Data Frame

Para comenzar la exploración, es importante conocer la estructura básica del DataFrame resultante. Esto incluye saber cuántas filas (registros) y columnas (atributos) contiene, lo cual nos da una idea del tamaño y la complejidad de los datos con los que vamos a trabajar.

In [7]:
print(f"El DataFrame tiene {df_res.shape[0]} filas y {df_res.shape[1]} columnas.") # 0 para filas, 1 para columnas

El DataFrame tiene 200 filas y 24 columnas.


### Mostramos las primeras 10 filas para un vistazo rápido a la información

Para iniciar el análisis exploratorio, es fundamental observar una muestra de los datos. Visualizamos las primeras filas del DataFrame

In [8]:
df_res.head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],...,41.884193,-87.647946,809 W Randolph,NaN,NaN,Chicago,60607,US,IL,"['809 W Randolph', 'Chicago, IL 60607']"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",...,41.890694,-87.624782,444 N Michigan Ave,NaN,NaN,Chicago,60611,US,IL,"['444 N Michigan Ave', 'Chicago, IL 60611']"
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",...,41.924459,-87.710898,3309 W Fullerton Ave,NaN,NaN,Chicago,60647,US,IL,"['3309 W Fullerton Ave', 'Chicago, IL 60647']"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],...,41.881689,-87.625006,12 S Michigan Ave,NaN,NaN,Chicago,60603,US,IL,"['12 S Michigan Ave', 'Chicago, IL 60603']"
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",...,41.886944,-87.648795,302 N Green St,Fl 3,NaN,Chicago,60607,US,IL,"['302 N Green St', 'Fl 3', 'Chicago, IL 60607']"
5,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,False,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],...,41.889390,-87.635240,226 W Kinzie,NaN,NaN,Chicago,60654,US,IL,"['226 W Kinzie', 'Chicago, IL 60654']"
6,cgIuo3geaxw1Sfhbqm5qTA,alla-vita-chicago-4,Alla Vita,https://s3-media0.fl.yelpcdn.com/bphoto/X4XMxK...,False,https://www.yelp.com/biz/alla-vita-chicago-4?a...,741,"[{'alias': 'italian', 'title': 'Italian'}]",4.4,"['delivery', 'pickup']",...,41.884680,-87.642322,564 W Randolph St,NaN,NaN,Chicago,60661,US,IL,"['564 W Randolph St', 'Chicago, IL 60661']"
7,PZe0q_153VHUnaR-8dOTJg,the-dearborn-chicago-2,The Dearborn,https://s3-media0.fl.yelpcdn.com/bphoto/eSXeGi...,False,https://www.yelp.com/biz/the-dearborn-chicago-...,2603,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,"['delivery', 'pickup']",...,41.884253,-87.629315,145 N Dearborn St,NaN,NaN,Chicago,60602,US,IL,"['145 N Dearborn St', 'Chicago, IL 60602']"
8,1BHgaf25FpVSOXFpZF0pYw,trivoli-tavern-chicago,Trivoli Tavern,https://s3-media0.fl.yelpcdn.com/bphoto/kMNBfN...,False,https://www.yelp.com/biz/trivoli-tavern-chicag...,839,"[{'alias': 'tradamerican', 'title': 'American'}]",4.5,['delivery'],...,41.883722,-87.649039,114 N Green St,NaN,NaN,Chicago,60607,US,IL,"['114 N Green St', 'Chicago, IL 60607']"
9,xFBQ1md6PDm7YEpJARoxAA,the-vig-chicago-chicago-2,The VIG Chicago,https://s3-media0.fl.yelpcdn.com/bphoto/LTJQDX...,False,https://www.yelp.com/biz/the-vig-chicago-chica...,905,"[{'alias': 'tradamerican', 'title': 'American'...",4.1,"['delivery', 'pickup']",...,41.909925,-87.634503,1527 N Wells St,NaN,NaN,Chicago,60610,US,IL,"['1527 N Wells St', 'Chicago, IL 60610']"


### Listamos el nombre y la cantidad total de columnas del DataFrame

Conocer el nombre y la cantidad de columnas es esencial para tener una visión clara de la estructura de nuestro DataFrame

In [9]:
for i, column in enumerate(df_res.columns, 1):
    print(f"{i} : {column}")

1 : id
2 : alias
3 : name
4 : image_url
5 : is_closed
6 : url
7 : review_count
8 : categories
9 : rating
10 : transactions
11 : price
12 : phone
13 : display_phone
14 : distance
15 : coordinates.latitude
16 : coordinates.longitude
17 : location.address1
18 : location.address2
19 : location.address3
20 : location.city
21 : location.zip_code
22 : location.country
23 : location.state
24 : location.display_address


### Es importante identificar cuáles columnas son numéricas y cuáles son categóricas, ya que esto nos permitirá anticipar qué funciones o herramientas utilizar según el tipo de dato que queramos analizar o transformar

In [10]:
# Identificar columnas categóricas

columnas_categoricas = df_res.select_dtypes(include='object').columns.tolist() # Clasifica por tipo de dato y devuelve una lista, object para cadenas de texto
print("Columnas categóricas:", columnas_categoricas)
print("Número de columnas categóricas:", len(columnas_categoricas))

# Identificar columnas numéricas

columnas_numericas = df_res.select_dtypes(include='number').columns.tolist() # number para tipos numéricos
print("Columnas numéricas:", columnas_numericas)
print("Número de columnas numéricas:", len(columnas_numericas))

Columnas categóricas: ['id', 'alias', 'name', 'image_url', 'url', 'categories', 'transactions', 'price', 'display_phone', 'location.address1', 'location.address2', 'location.address3', 'location.city', 'location.country', 'location.state', 'location.display_address']
Número de columnas categóricas: 16
Columnas numéricas: ['review_count', 'rating', 'phone', 'distance', 'coordinates.latitude', 'coordinates.longitude', 'location.zip_code']
Número de columnas numéricas: 7


### Una vez que conocemos los datos generales, valores y categorías con los que estaremos trabajando, es útil mostrar información más técnica del DataFrame. Esto nos ayudará a identificar qué columnas o valores requieren limpieza y normalización para futuros análisis más precisos

In [11]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        200 non-null    object 
 1   alias                     200 non-null    object 
 2   name                      200 non-null    object 
 3   image_url                 200 non-null    object 
 4   is_closed                 200 non-null    bool   
 5   url                       200 non-null    object 
 6   review_count              200 non-null    int64  
 7   categories                200 non-null    object 
 8   rating                    200 non-null    float64
 9   transactions              200 non-null    object 
 10  price                     133 non-null    object 
 11  phone                     186 non-null    float64
 12  display_phone             186 non-null    object 
 13  distance                  200 non-null    float64
 14  coordinate

### Utilizamos una función de la librería Pandas que nos permite obtener información técnica y estadística detallada del DataFrame. Esto facilita la detección de anomalías o problemas específicos en cada categoría, ayudando a enfocar los esfuerzos de limpieza y análisis.

In [12]:
df_res.describe(include='all')

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
count,200,200,200,200,200,200,200.000000,200,200.000000,200,...,200.000000,200.000000,199,9,2,200,200.000000,200,200,200
unique,200,200,200,200,1,200,NaN,180,NaN,9,...,NaN,NaN,197,8,2,2,NaN,1,1,198
top,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,NaN,"[{'alias': 'italian', 'title': 'Italian'}]",NaN,"['delivery', 'pickup']",...,NaN,NaN,59 W Hubbard St,Lower Level,Willis Tower,Chicago,NaN,US,IL,"['120 N Wacker Dr', 'Chicago, IL 60606']"
freq,1,1,1,1,200,1,NaN,8,NaN,97,...,NaN,NaN,2,2,1,199,NaN,200,200,2
mean,NaN,NaN,NaN,NaN,NaN,NaN,844.550000,NaN,4.396000,NaN,...,41.905619,-87.659526,NaN,NaN,NaN,NaN,60626.765000,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,1615.011212,NaN,0.235502,NaN,...,0.026860,0.027903,NaN,NaN,NaN,NaN,18.776918,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,3.800000,NaN,...,41.846210,-87.720520,NaN,NaN,NaN,NaN,60601.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,84.750000,NaN,4.300000,NaN,...,41.887617,-87.679181,NaN,NaN,NaN,NaN,60611.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,283.000000,NaN,4.400000,NaN,...,41.902247,-87.654876,NaN,NaN,NaN,NaN,60622.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,762.000000,NaN,4.500000,NaN,...,41.920242,-87.633773,NaN,NaN,NaN,NaN,60647.000000,NaN,NaN,NaN


---------------------

### Principales hallazgos tras la exploración inicial del DataFrame de restaurantes en Chicago

Después de analizar el DataFrame obtenido de la API de Yelp, se identificaron los siguientes puntos clave para orientar el proceso de limpieza y transformación:

- **Tamaño del dataset:** El DataFrame cuenta con 200 filas y 24 columnas, lo que proporciona una muestra adecuada para análisis exploratorios y posteriores modelados.
- **Tipos de variables:** Se detectaron 16 columnas categóricas y 7 columnas numéricas, permitiendo realizar tanto análisis descriptivos como inferenciales.
- **Valores nulos:** Varias columnas presentan valores faltantes, especialmente `price`, `phone` e `image_url`. Destaca que la columna `price` tiene cerca del 35% de valores nulos, lo que puede afectar la calidad del análisis si no se trata adecuadamente.
- **Redundancia de información:** Existen columnas con información duplicada, como las diferentes variantes de dirección (`location.address1`, `location.address2`, `location.address3`, `location.display_address`) y teléfono (`phone`, `display_phone`). Esto sugiere la necesidad de unificar y depurar estas variables para evitar inconsistencias.
- **Tipos de datos incorrectos:** Algunas columnas no tienen el tipo de dato adecuado. Por ejemplo, `phone` aparece como numérico cuando debería ser texto, y columnas como `categories` y `transactions` están en formato string representando listas o diccionarios.
- **Columnas irrelevantes:** Algunas variables, como `distance` e `is_closed`, no aportan información útil para el análisis y pueden eliminarse para simplificar el DataFrame.
- **Transformación de variables:** La columna `price` utiliza símbolos (`$`, `$$`, etc.) para representar rangos de precios, por lo que es necesario transformarla a valores numéricos para facilitar el análisis.
- **Información valiosa en categorías:** La columna `categories` contiene datos relevantes sobre el tipo de restaurante, pero requiere transformación para ser utilizada eficientemente en análisis posteriores.
- **Imputación inteligente:** Es fundamental imputar los valores faltantes, especialmente en `price`, de manera informada para no perder información relevante ni introducir sesgos en los resultados.

---

**Conclusión:**  
La exploración inicial permitió identificar los principales retos y oportunidades en la limpieza y transformación de los datos. El siguiente paso será abordar estos puntos mediante técnicas de depuración, imputación y transformación de variables para asegurar la calidad y utilidad del dataset en los análisis posteriores.

----------------------------

## **Limpieza de datos**

#### Identificación y tratamiento de valores nulos, duplicados y errores

Después de la exploración inicial, es fundamental depurar el DataFrame para garantizar la calidad y consistencia de los datos. En esta sección se realizarán las siguientes acciones:

- **Manejo de valores nulos y fuera de rango:** Se imputarán los datos faltantes considerando las categorías relevantes, para mantener la integridad de la información y evitar sesgos en los resultados.
- **Revisión de registros duplicados:** Se analizará si existen filas duplicadas y se decidirá la mejor estrategia para tratarlas, ya sea eliminarlas o consolidarlas.
- **Corrección de tipos de datos:** Se ajustarán los tipos de datos de las columnas para que sean coherentes con la información que representan.
- **Unificación de columnas redundantes:** Se combinarán columnas que contienen información similar o repetida para simplificar el DataFrame y facilitar su análisis.
---------------------------

----
# Índice

- [1. Conexión a la API de Yelp](#Conexion-a-la-API-de-YELP)
- [2. Análisis exploratorio de datos](#Exploracion-del-DataFrame)
- [3. Limpieza de datos](#Limpieza-de-datos)
- [4. Conclusiones](#Conclusiones-generales)
--------

### Iniciamos la etapa de limpieza verificando la presencia de valores nulos o duplicados en la columna 'id'

Aunque en la exploración inicial no se detectaron valores nulos en la columna `id`, es recomendable realizar una segunda revisión para asegurarnos de que cada registro cuente con un identificador único y válido. Esto es fundamental para evitar problemas de integridad en el DataFrame y garantizar que no existan registros duplicados o sin identificar.

In [13]:
# Identificar valores nulos 

id_ducplicado = df_res['id'][df_res['id'].duplicated()] # encontrar ids duplicados 
id_ducplicado 

Series([], Name: id, dtype: object)

### Tras confirmar que no existen valores nulos en la columna 'id', revisamos los valores únicos en 'location.city'

Aunque la mayoría de los registros tienen la ciudad correctamente identificada como "Chicago", observamos que existen algunos valores diferentes en la columna `location.city`. Al analizar estos casos, comprobamos que, a pesar de las diferencias en el nombre, todos los registros corresponden a la misma ciudad. Esto puede deberse a variaciones en la escritura o a errores menores en la fuente de datos, pero no afecta la integridad del análisis, ya que todos pertenecen al área metropolitana de Chicago.

In [14]:
# calcular valores unicos por columna

valores_unicos = df_res['location.city'].value_counts() # Esto devuelve un array de valores únicos en la columna 'Chicago', si existe. Si no, cambiar por una columna existente.
  
print("Valores únicos por columna:")
print(valores_unicos) 

Valores únicos por columna:
location.city
Chicago         199
Lincoln Park      1
Name: count, dtype: int64


### Posteriormente, identificamos columnas con información redundante, especialmente en las direcciones. Por ello, unificamos 'location.address1', 'location.address2' y 'location.address3' en una sola columna

Durante la exploración detectamos que las columnas de dirección estaban separadas en varios campos, lo que complica el análisis y genera redundancia. Para simplificar y estandarizar la información, combinamos estas tres columnas en una sola llamada `address`. Así, la dirección completa de cada restaurante queda en un solo campo, facilitando futuras consultas y análisis. Si en algún momento se requiere separar nuevamente la información, es posible hacerlo, pero para el análisis actual resulta más eficiente trabajar con una sola columna.

In [15]:
# Juntar columnas de adress (1,2,3) en una sola columna

df_res['address'] = df_res[['location.address1', 'location.address2', 'location.address3']].apply(lambda x: ', '.join(x.dropna()), axis=1) # Combina las columnas de dirección en una sola, eliminando valores nulos de las columnas originales donde no hay datos

df_res.drop(columns=['location.address1', 'location.address2', 'location.address3'], inplace=True) # Elimina las columnas originales de dirección para evitar redundancia

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,display_phone,distance,coordinates.latitude,coordinates.longitude,location.city,location.zip_code,location.country,location.state,location.display_address,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],...,(312) 492-6262,3401.238676,41.884193,-87.647946,Chicago,60607,US,IL,"['809 W Randolph', 'Chicago, IL 60607']",809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",...,(312) 464-1744,4672.658118,41.890694,-87.624782,Chicago,60611,US,IL,"['444 N Michigan Ave', 'Chicago, IL 60611']",444 N Michigan Ave
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",...,(773) 772-2343,3480.792884,41.924459,-87.710898,Chicago,60647,US,IL,"['3309 W Fullerton Ave', 'Chicago, IL 60647']",3309 W Fullerton Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],...,(312) 792-3502,5091.190048,41.881689,-87.625006,Chicago,60603,US,IL,"['12 S Michigan Ave', 'Chicago, IL 60603']",12 S Michigan Ave
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",...,(773) 645-1400,3120.144506,41.886944,-87.648795,Chicago,60607,US,IL,"['302 N Green St', 'Fl 3', 'Chicago, IL 60607']","302 N Green St, Fl 3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,Crisp,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,False,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",...,(773) 697-7610,4401.590420,41.936121,-87.644421,Chicago,60657,US,IL,"['2940 N Broadway Ave', 'Chicago, IL 60657']",2940 N Broadway Ave
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,Sapori Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,False,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",...,(773) 832-9999,3800.150974,41.931660,-87.648590,Chicago,60614,US,IL,"['2701 N Halsted St', 'Chicago, IL 60614']",2701 N Halsted St
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,Bad Habit,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,False,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],...,(773) 900-2449,1805.650848,41.918715,-87.689922,Chicago,60647,US,IL,"['2047 N Milwaukee Ave', 'Chicago, IL 60647']",2047 N Milwaukee Ave
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,Daebak,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,False,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",...,(312) 722-6923,729.507821,41.904660,-87.668940,Chicago,60622,US,IL,"['1266 N Milwaukee Ave', 'Chicago, IL 60622']",1266 N Milwaukee Ave


### Seguimos con la eliminación de columnas que tienen redundancia de información. Notamos que la información que se maneja en 'location.display_address' y 'address' es muy similar, pero con ligeras diferencias de formato o detalle. Para decidir con cuál quedarnos, utilizamos la librería `fuzzywuzzy`, que permite comparar cadenas de texto y obtener un porcentaje de similitud entre ambas columnas.

Este análisis nos ayuda a identificar si realmente ambas columnas aportan la misma información o si alguna contiene datos únicos relevantes. Si la similitud es alta, optamos por conservar solo una columna (en este caso, 'address') para simplificar el DataFrame y evitar redundancias. A continuación, se detalla el proceso de comparación y los criterios utilizados para la depuración.

El primer paso consiste en crear una nueva columna llamada `similitud_direccion`, donde se almacena el porcentaje de similitud entre los valores de `'location.display_address'` y `'address'` para cada fila. Para calcular este porcentaje, se utiliza la función `fuzz.ratio` de la librería `fuzzywuzzy`, que compara ambas cadenas y devuelve un valor entre 0 y 100. Así, podemos identificar fácilmente qué tan parecidas son las direcciones en ambas columnas. A continuación, se muestran las primeras 5 filas para verificar que el cálculo se realizó correctamente.

In [16]:
# Crear una nueva columna con el porcentaje de similitud entre las dos columnas de dirección

df_res['similitud_direccion'] = df_res.apply( # aplica una función a lo largo de un eje del DataFrame
    lambda row: fuzz.ratio(str(row['location.display_address']), str(row['address'])), # función lambda que calcula el porcentaje de similitud entre las dos columnas de dirección para cada fila
    axis=1 # especifica que la función se aplica a lo largo de las filas (axis=1)
)

print(df_res[['location.display_address', 'address', 'similitud_direccion']].head()) # Mostrar las primeras 5 filas de las columnas relevantes para verificar los resultados


                          location.display_address               address  \
0          ['809 W Randolph', 'Chicago, IL 60607']        809 W Randolph   
1      ['444 N Michigan Ave', 'Chicago, IL 60611']    444 N Michigan Ave   
2    ['3309 W Fullerton Ave', 'Chicago, IL 60647']  3309 W Fullerton Ave   
3       ['12 S Michigan Ave', 'Chicago, IL 60603']     12 S Michigan Ave   
4  ['302 N Green St', 'Fl 3', 'Chicago, IL 60607']  302 N Green St, Fl 3   

   similitud_direccion  
0                   53  
1                   59  
2                   62  
3                   58  
4                   60  


Ya que tenemos esto, el siguiente paso es revisar los casos donde el porcentaje de similitud es menor al 50%. Consideramos este umbral adecuado debido a las diferencias de formato entre ambas columnas. Aquellas filas con un porcentaje menor a 50% se revisan manualmente, ya que generalmente esto ocurre cuando el valor de 'address' es más corto que el de 'location.display_address' (por ejemplo, cuando la dirección es muy breve o incompleta). Así, nos aseguramos de no eliminar información relevante por diferencias de formato.

In [17]:
# Crear otra columna con un valor booleano que indique si la similitud es menor al 50%

df_res['similitud_direccion_menor_50'] = df_res['similitud_direccion'] < 50 # Crear una nueva columna que indique si la similitud es menor al 50%

df_res['similitud_direccion_menor_50'].value_counts() # Contar cuántas filas tienen similitud menor al 50%

similitud_direccion_menor_50
False    195
True       5
Name: count, dtype: int64

In [18]:
df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.city,location.zip_code,location.country,location.state,location.display_address,address,similitud_direccion,similitud_direccion_menor_50
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],...,41.884193,-87.647946,Chicago,60607,US,IL,"['809 W Randolph', 'Chicago, IL 60607']",809 W Randolph,53,False
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",...,41.890694,-87.624782,Chicago,60611,US,IL,"['444 N Michigan Ave', 'Chicago, IL 60611']",444 N Michigan Ave,59,False
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",...,41.924459,-87.710898,Chicago,60647,US,IL,"['3309 W Fullerton Ave', 'Chicago, IL 60647']",3309 W Fullerton Ave,62,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],...,41.881689,-87.625006,Chicago,60603,US,IL,"['12 S Michigan Ave', 'Chicago, IL 60603']",12 S Michigan Ave,58,False
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",...,41.886944,-87.648795,Chicago,60607,US,IL,"['302 N Green St', 'Fl 3', 'Chicago, IL 60607']","302 N Green St, Fl 3",60,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,Crisp,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,False,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",...,41.936121,-87.644421,Chicago,60657,US,IL,"['2940 N Broadway Ave', 'Chicago, IL 60657']",2940 N Broadway Ave,60,False
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,Sapori Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,False,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",...,41.931660,-87.648590,Chicago,60614,US,IL,"['2701 N Halsted St', 'Chicago, IL 60614']",2701 N Halsted St,58,False
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,Bad Habit,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,False,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],...,41.918715,-87.689922,Chicago,60647,US,IL,"['2047 N Milwaukee Ave', 'Chicago, IL 60647']",2047 N Milwaukee Ave,62,False
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,Daebak,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,False,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",...,41.904660,-87.668940,Chicago,60622,US,IL,"['1266 N Milwaukee Ave', 'Chicago, IL 60622']",1266 N Milwaukee Ave,62,False


Ya que hemos decidido quedarnos únicamente con la columna 'address' (pues la información relevante de las demás columnas ya está contenida en otras variables del DataFrame), el siguiente paso es eliminar todas aquellas columnas que resultaron redundantes o que solo se utilizaron para la comparación, así como las que no aportan valor al análisis. Esto permite simplificar el DataFrame y dejar únicamente la información necesaria para los análisis posteriores.

In [19]:
# Eliminar columnas innecesarias

df_res.drop(columns=['similitud_direccion_menor_50', 'similitud_direccion', 'location.display_address', 'location.state', 'location.country', 'location.city'], inplace=True) # Eliminamos las columnas que ya no son necesarias, incluyendo las que se usaron para la comparación y las que no aportan información relevante

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,1.312493e+10,(312) 492-6262,3401.238676,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,1.312464e+10,(312) 464-1744,4672.658118,41.890694,-87.624782,60611,444 N Michigan Ave
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,1.773772e+10,(773) 772-2343,3480.792884,41.924459,-87.710898,60647,3309 W Fullerton Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,1.312792e+10,(312) 792-3502,5091.190048,41.881689,-87.625006,60603,12 S Michigan Ave
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,1.773645e+10,(773) 645-1400,3120.144506,41.886944,-87.648795,60607,"302 N Green St, Fl 3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,Crisp,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,False,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",$$,1.773698e+10,(773) 697-7610,4401.590420,41.936121,-87.644421,60657,2940 N Broadway Ave
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,Sapori Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,False,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",$$$,1.773833e+10,(773) 832-9999,3800.150974,41.931660,-87.648590,60614,2701 N Halsted St
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,Bad Habit,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,False,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],NaN,1.773900e+10,(773) 900-2449,1805.650848,41.918715,-87.689922,60647,2047 N Milwaukee Ave
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,Daebak,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,False,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",$$$,1.312723e+10,(312) 722-6923,729.507821,41.904660,-87.668940,60622,1266 N Milwaukee Ave


### Procedemos a hacer lo mismo con las columnas 'display_phone' y 'phone'

Al igual que con las columnas de dirección, analizamos la similitud entre 'display_phone' y 'phone' utilizando la librería `fuzzywuzzy`. Esto nos permite identificar si ambas columnas contienen la misma información o si existen diferencias relevantes. Si la similitud es alta en la mayoría de los casos, optamos por conservar solo una de ellas (en este caso, 'phone') y eliminar la otra junto con las columnas auxiliares generadas para la comparación, simplificando así el DataFrame.

In [20]:
# Crear una nueva columna con el porcentaje de similitud entre las dos columnas de teléfono

df_res['similitud_tel'] = df_res.apply( # aplica una función a lo largo de un eje del DataFrame
    lambda row: fuzz.ratio(str(row['phone']), str(row['display_phone'])), # función lambda que calcula el porcentaje de similitud entre las dos columnas de teléfono para cada fila
    axis=1 # especifica que la función se aplica a lo largo de las filas (axis=1
)

print(df_res[['phone', 'display_phone', 'similitud_tel']].head()) # Mostrar las primeras 5 filas de las columnas relevantes para verificar los resultados

          phone   display_phone  similitud_tel
0  1.312493e+10  (312) 492-6262             74
1  1.312464e+10  (312) 464-1744             74
2  1.773772e+10  (773) 772-2343             74
3  1.312792e+10  (312) 792-3502             74
4  1.773645e+10  (773) 645-1400             74


Verificamos cuáles registros tienen una similitud menor al 50% entre 'display_phone' y 'phone'. Sin embargo, observamos que todos los valores presentan una similitud mayor al 70%, lo que indica que ambas columnas contienen prácticamente la misma información, solo con ligeras diferencias de formato.

In [21]:
# Saber los que son menores a 50%

df_res['similitud_tel_menor_50'] = df_res['similitud_tel'] < 50 # Crear una nueva columna que indique si la similitud es menor al 50%

df_res['similitud_tel_menor_50'].value_counts() # Contar cuántas filas tienen similitud menor al 50%


similitud_tel_menor_50
False    200
Name: count, dtype: int64

Con la confirmación anterior de que todas las columnas presentan una similitud mayor al 70%, procedemos a eliminar las columnas innecesarias y nos quedamos únicamente con 'phone'. Esto permite simplificar el DataFrame y evitar redundancias, ya que ambas columnas contienen prácticamente la misma información.

In [22]:
# eliminar columnas innecesarias

df_res.drop(columns=['similitud_tel', 'display_phone', 'similitud_tel_menor_50'], inplace=True) # se eliminan las columnas que ya no son necesarias

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,distance,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,1.312493e+10,3401.238676,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,1.312464e+10,4672.658118,41.890694,-87.624782,60611,444 N Michigan Ave
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,1.773772e+10,3480.792884,41.924459,-87.710898,60647,3309 W Fullerton Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,1.312792e+10,5091.190048,41.881689,-87.625006,60603,12 S Michigan Ave
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,1.773645e+10,3120.144506,41.886944,-87.648795,60607,"302 N Green St, Fl 3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,Crisp,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,False,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",$$,1.773698e+10,4401.590420,41.936121,-87.644421,60657,2940 N Broadway Ave
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,Sapori Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,False,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",$$$,1.773833e+10,3800.150974,41.931660,-87.648590,60614,2701 N Halsted St
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,Bad Habit,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,False,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],NaN,1.773900e+10,1805.650848,41.918715,-87.689922,60647,2047 N Milwaukee Ave
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,Daebak,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,False,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",$$$,1.312723e+10,729.507821,41.904660,-87.668940,60622,1266 N Milwaukee Ave


### También cambiamos el tipo de dato de 'phone', ya que este se encuentra en formato 'float'. Para ello, primero imputamos los valores nulos con '0' para evitar errores al convertir el tipo de dato. Posteriormente, transformamos la columna a tipo entero ('int') para eliminar los decimales y finalmente la convertimos a tipo cadena ('str'), ya que no se realizarán operaciones matemáticas con estos datos.

Primero imputamos aquellos valores nulos con '0', ya que si la columna no está completa al momento de cambiar el tipo de dato, se generará un error. Así aseguramos que todos los registros tengan un valor y el cambio de tipo se realice correctamente.

In [23]:
# Cambiar tipo de valor en phone, borrar o imputar los nulos antes

df_res['phone'] = df_res['phone'].fillna('0') # Imputar los valores nulos con '0' para evitar errores al cambiar el tipo de dato
df_res.info() # Verificar el cambio de tipo de dato en la columna 'phone' 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   name                   200 non-null    object 
 3   image_url              200 non-null    object 
 4   is_closed              200 non-null    bool   
 5   url                    200 non-null    object 
 6   review_count           200 non-null    int64  
 7   categories             200 non-null    object 
 8   rating                 200 non-null    float64
 9   transactions           200 non-null    object 
 10  price                  133 non-null    object 
 11  phone                  200 non-null    object 
 12  distance               200 non-null    float64
 13  coordinates.latitude   200 non-null    float64
 14  coordinates.longitude  200 non-null    float64
 15  locati

Cuando ya no tenemos valores nulos, convertimos los datos a tipo entero (`int`). Esto elimina los puntos decimales que puedan existir debido al formato original 'float' y deja la columna lista para su posterior conversión a cadena de texto.

In [24]:
df_res['phone'] = df_res['phone'].astype(int) # Cambio a 'int''

df_res.info() # Verificar el cambio de tipo de dato en la columna 'phone'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   name                   200 non-null    object 
 3   image_url              200 non-null    object 
 4   is_closed              200 non-null    bool   
 5   url                    200 non-null    object 
 6   review_count           200 non-null    int64  
 7   categories             200 non-null    object 
 8   rating                 200 non-null    float64
 9   transactions           200 non-null    object 
 10  price                  133 non-null    object 
 11  phone                  200 non-null    int64  
 12  distance               200 non-null    float64
 13  coordinates.latitude   200 non-null    float64
 14  coordinates.longitude  200 non-null    float64
 15  locati

Terminamos cambiando el tipo de dato a 'str', ya que no se realizarán operaciones matemáticas con estos datos y es más adecuado tratarlos como texto.

In [25]:
df_res['phone'] = df_res['phone'].astype(str) # Cambio a 'str'

df_res.info() # Verificar el cambio de tipo de dato en la columna 'phone'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   name                   200 non-null    object 
 3   image_url              200 non-null    object 
 4   is_closed              200 non-null    bool   
 5   url                    200 non-null    object 
 6   review_count           200 non-null    int64  
 7   categories             200 non-null    object 
 8   rating                 200 non-null    float64
 9   transactions           200 non-null    object 
 10  price                  133 non-null    object 
 11  phone                  200 non-null    object 
 12  distance               200 non-null    float64
 13  coordinates.latitude   200 non-null    float64
 14  coordinates.longitude  200 non-null    float64
 15  locati

### Seguimos con la eliminación de columnas. Observamos la columna 'distance' y decidimos investigar a qué hacía referencia. Descubrimos que indica la distancia entre el lugar donde se generó la API Key y la ubicación del restaurante. Por esta razón, se tomó la decisión de eliminarla, ya que no aporta información relevante para el análisis.

In [26]:
df_res.drop(columns=['distance'], inplace=True) #lo borramos pq no tiene sentido, la distancia es entre el negocio y de donde se realizó la consulta (la key)
df_res

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,Crisp,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,False,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,Sapori Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,False,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,Bad Habit,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,False,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,Daebak,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,False,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave


### También, gracias a la primera fase de exploración, notamos que en la columna 'is_closed' solo había un valor repetido en todos los registros. Por ello, corroboramos esta información y posteriormente eliminamos la columna, ya que no aporta valor al análisis al contener siempre el mismo dato.

In [27]:
# contar los falses de is_closed

df_res['is_closed'].value_counts() # Contar los valores únicos en la columna 'is_closed' para ver cuántos son True y cuántos son False

is_closed
False    200
Name: count, dtype: int64

In [28]:
# Se elimina la columna is_closed

df_res.drop(columns=['is_closed'], inplace=True) 

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,Crisp,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,Sapori Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,Bad Habit,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,Daebak,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave


### Al igual que en instancias anteriores, notamos la similitud entre las columnas 'alias' y 'name', ambas referentes al nombre del restaurante. Por ello, aplicamos nuevamente nuestro método de comparación utilizando la librería 'fuzzywuzzy' para analizar el grado de coincidencia entre ambas columnas y decidir cuál conservar.

In [29]:
# Comparar alias y name

df_res['similitud_nombre'] = df_res.apply(
    lambda row: fuzz.ratio(str(row['alias']), str(row['name'])), # función lambda que calcula el porcentaje de similitud entre las dos columnas de nombre para cada fila
    axis=1
)

print(df_res[['alias', 'name', 'similitud_nombre']].head()) # Mostrar las primeras 5 filas de las columnas relevantes para verificar los resultados

                       alias             name  similitud_nombre
0  girl-and-the-goat-chicago  Girl & The Goat                40
1     the-purple-pig-chicago   The Purple Pig                50
2           penumbra-chicago         Penumbra                58
3     cindys-rooftop-chicago  Cindy's Rooftop                59
4              aba-chicago-2              Aba                25


Debido a que también sabemos que existen diferencias en el formato en el que están escritos, observamos que una mayor cantidad de registros presentan un porcentaje de similitud menor al 50%. Esto se debe a variaciones en la forma en que se almacenan los nombres en ambas columnas.

In [30]:
# Saber los que son menores a 50%

df_res['similitud_nombre_menor_50'] = df_res['similitud_nombre'] < 50 # Crear una nueva columna que indique si la similitud es menor al 50%

df_res['similitud_nombre_menor_50'].value_counts() # Contar cuántas filas tienen similitud menor al 50%

similitud_nombre_menor_50
False    141
True      59
Name: count, dtype: int64

In [31]:
df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,similitud_nombre,similitud_nombre_menor_50
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,40,True
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,50,False
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,58,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,59,False
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3",25,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,Crisp,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,44,True
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,Sapori Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,65,False
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,Bad Habit,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,46,True
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,Daebak,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,50,False


Debido a que hay muchos valores por debajo del 50%, se decide hacer una inspección visual de los datos para determinar si es necesario eliminar o modificar alguna fila en particular. Llegamos a la conclusión de que se trata de lo mismo pero en 'alias' hay mayor información, por lo que se decide eliminar la columna 'name' y quedarse con 'alias'.

In [32]:
df_res.drop(columns=['name', 'similitud_nombre', 'similitud_nombre_menor_50'], inplace=True) # eliminamos las columnas que ya no son necesarias

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave


### Una vez completado esto, pasamos a la categoría de 'categories'. Observamos que el tipo de dato de esta columna era un objeto, pero en realidad contenía principalmente cadenas de texto ('str') con formato de lista de diccionarios. Esto dificultaba el trabajo con las categorías de manera individual. Por ello, utilizamos la librería 'ast', que permite transformar estos datos de tipo string a objetos propios de Python (listas y diccionarios), facilitando así su manipulación y análisis en los siguientes pasos.

Empezamos por cambiar el tipo de dato de 'str' a 'lista', ya que originalmente era una cadena con formato de lista de diccionarios. Al aplicar la conversión con `ast.literal_eval`, ahora sí contamos con una lista real de diccionarios en cada fila, lo que facilita el manejo y análisis de las categorías de manera individual.

In [33]:
# Cambiar el tipo de dato de 'categories' de 'str' a 'lista'

df_res["categories"] = df_res["categories"].apply(ast.literal_eval)

In [34]:
df_res.head() # Mostramos el DataFrame actualizado para verificar los cambios


,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3"


Ahora extraemos los valores de las claves de los diccionarios que están en la lista de cada fila. Esto nos permite acceder de manera más sencilla a la información relevante de cada categoría y facilita la creación de nuevas columnas o el análisis individual de cada atributo dentro de las categorías.

In [35]:
# Extraer los valores de las claves de los diccionarios en la columna 'categories'

df_res["category_keys"] = df_res["categories"].apply(lambda lst: [list(d.values()) for d in lst]) # Extrae las claves de los diccionarios en la lista y los guarda en una nueva columna 'category_keys'


In [36]:
df_res.head() # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,category_keys
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[[newamerican, New American], [bars, Bars], [b..."
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[[tapasmallplates, Tapas/Small Plates], [medit..."
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[[wine_bars, Wine Bars], [seafood, Seafood], [..."
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[[newamerican, New American], [seafood, Seafoo..."
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[[mediterranean, Mediterranean], [tapasmallpla..."


Una vez extraídos estos datos, procedemos a tomar únicamente el primer valor de cada sublista, ya que por la forma en que se realizó la extracción, todos hacen referencia a lo mismo. De esta manera, los alias contienen la información en un formato más limpio y manejable, facilitando su uso en análisis posteriores.

In [37]:
# Sacar los primeros elementos de cada sublista 

df_res["aliases"] = df_res["category_keys"].apply(lambda lst: [sub[0] for sub in lst]) # Extrae los primeros elementos de cada sublista en 'category_keys' y los guarda en una nueva columna 'aliases'


In [38]:
df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,category_keys,aliases
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[[newamerican, New American], [bars, Bars], [b...","[newamerican, bars, bakeries]"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[[tapasmallplates, Tapas/Small Plates], [medit...","[tapasmallplates, mediterranean, newamerican]"
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[[wine_bars, Wine Bars], [seafood, Seafood], [...","[wine_bars, seafood, steak]"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[[newamerican, New American], [seafood, Seafoo...","[newamerican, seafood, breakfast_brunch]"
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[[mediterranean, Mediterranean], [tapasmallpla...","[mediterranean, tapasmallplates]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"['delivery', 'pickup']",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[[korean, Korean], [chicken_wings, Chicken Win...","[korean, chicken_wings]"
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.4,"['delivery', 'pickup']",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[[italian, Italian], [desserts, Desserts], [co...","[italian, desserts, cocktailbars]"
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[[cocktailbars, Cocktail Bars], [burgers, Burg...","[cocktailbars, burgers]"
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"['delivery', 'pickup']",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[[korean, Korean], [bbq, Barbeque]]","[korean, bbq]"


Ya que tenemos las categorías en un formato más manejable, eliminamos las columnas que ya no consideramos necesarias para evitar redundancia y dejar el DataFrame más limpio y enfocado solo en la información relevante para el análisis.

In [39]:
df_res.drop(columns=['category_keys', 'categories'], inplace=True) # eliminamos las columnas que ya no son necesarias

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,4.3,"['delivery', 'pickup']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]"
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,4.8,"['restaurant_reservation', 'delivery']",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[wine_bars, seafood, steak]"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]"
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,4.4,"['delivery', 'pickup']",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[mediterranean, tapasmallplates]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,4.3,"['delivery', 'pickup']",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[korean, chicken_wings]"
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,4.4,"['delivery', 'pickup']",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[italian, desserts, cocktailbars]"
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[cocktailbars, burgers]"
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,4.0,"['delivery', 'pickup']",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[korean, bbq]"


### De forma similar al proceso realizado con la columna de categorías, procedemos a cambiar el tipo de dato de la columna 'transactions', ya que originalmente es un string con formato de lista y no una lista real de Python. Esto permitirá manipular y analizar las transacciones de manera más eficiente en los siguientes pasos.

In [40]:
df_res["transactions"] = df_res["transactions"].apply(ast.literal_eval) # Cambiar el tipo de dato de 'transactions' de 'str' a 'lista'


In [41]:
df_res.describe(include='all') # Revisamos el cambio de tipo de dato en la columna 'transactions'

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases
count,200,200,200,200,200.000000,200.000000,200,133,200,200.000000,200.000000,200.000000,200,200
unique,200,200,200,200,NaN,NaN,9,4,186,NaN,NaN,NaN,198,180
top,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,NaN,NaN,"[delivery, pickup]",$$,0,NaN,NaN,NaN,120 N Wacker Dr,[italian]
freq,1,1,1,1,NaN,NaN,97,72,14,NaN,NaN,NaN,2,8
mean,NaN,NaN,NaN,NaN,844.550000,4.396000,NaN,NaN,NaN,41.905619,-87.659526,60626.765000,NaN,NaN
std,NaN,NaN,NaN,NaN,1615.011212,0.235502,NaN,NaN,NaN,0.026860,0.027903,18.776918,NaN,NaN
min,NaN,NaN,NaN,NaN,4.000000,3.800000,NaN,NaN,NaN,41.846210,-87.720520,60601.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,84.750000,4.300000,NaN,NaN,NaN,41.887617,-87.679181,60611.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,283.000000,4.400000,NaN,NaN,NaN,41.902247,-87.654876,60622.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,762.000000,4.500000,NaN,NaN,NaN,41.920242,-87.633773,60647.000000,NaN,NaN


### Prosiguiendo con el tratamiento de valores que pueden causar problemas, notamos que existe un valor nulo en la columna 'image_url'. Dado que estamos trabajando para una empresa cuyo principal enfoque es el marketing digital, consideramos eliminar este único registro, ya que las imágenes son fundamentales en este ámbito y la pérdida de un solo dato no afecta significativamente el análisis.

In [42]:
# Eliminar filas con valores nulos en image_url

df_res = df_res.dropna(subset=['image_url']) # Eliminar filas donde 'image_url' es nulo
df_res.info() # Verificar que no haya valores nulos en 'image_url' después de la eliminación, lo que hace que en total haya 199 filas en total

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   image_url              200 non-null    object 
 3   url                    200 non-null    object 
 4   review_count           200 non-null    int64  
 5   rating                 200 non-null    float64
 6   transactions           200 non-null    object 
 7   price                  133 non-null    object 
 8   phone                  200 non-null    object 
 9   coordinates.latitude   200 non-null    float64
 10  coordinates.longitude  200 non-null    float64
 11  location.zip_code      200 non-null    int64  
 12  address                200 non-null    object 
 13  aliases                200 non-null    object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.0+ KB


### Para finalizar la limpieza de este DataFrame, nos enfocamos en la columna 'price', donde observamos que existen 70 valores nulos. Es necesario imputar estos valores, ya que al tratarse de un DataFrame pequeño, eliminar o ignorar cerca del 35% de los datos supondría un riesgo para futuros análisis. A continuación, se detallan los pasos realizados para el tratamiento de estos valores faltantes.

Comenzamos identificando los valores nulos presentes en la columna 'price', así como su tabla de frecuencia (sin considerar los valores nulos). Esto nos permite conocer la distribución de los precios existentes y tener una base para decidir la mejor estrategia de imputación para los valores faltantes.

In [43]:
# Revisar los valores únicos en la columna 'price'

df_res['price'].unique()

array(['$$$', '$$', nan, '$$$$', '$'], dtype=object)

In [44]:
# Tabla de frecuencia de los valores en la columna 'price'

conteo_valores = df_res['price'].value_counts() # Calcula la frecuencia de cada valor único en la columna 'price'

conteo_valores # Mostrar la tabla de frecuencia de los valores en la columna 'price'

price
$$      72
$$$     46
$$$$    14
$        1
Name: count, dtype: int64

Para conocer la diversidad de categorías presentes en el dataset, aplanamos la lista de aliases de todos los restaurantes y obtenemos los valores únicos. Esto nos permite identificar cuántas y cuáles categorías distintas existen en la columna `aliases`:

In [45]:
# Aplanar la lista de aliases y obtener los valores únicos
import itertools

todos_aliases = list(itertools.chain.from_iterable(df_res['aliases']))
valores_unicos = set(todos_aliases)
print(f"Número de aliases únicos: {len(valores_unicos)}")
print(valores_unicos)

Número de aliases únicos: 101
{'newamerican', 'beer_and_wine', 'cafes', 'cambodian', 'vegetarian', 'mexican', 'brasseries', 'comfortfood', 'chinese', 'korean', 'beerbar', 'hainan', 'desserts', 'gastropubs', 'steak', 'vietnamese', 'lebanese', 'panasian', 'japacurry', 'waffles', 'noodles', 'bbq', 'caribbean', 'argentine', 'tacos', 'coffee', 'southern', 'supperclubs', 'dimsum', 'greek', 'salad', 'venezuelan', 'pastashops', 'wine_bars', 'mideastern', 'lounges', 'breweries', 'diners', 'hotpot', 'italian', 'ramen', 'restaurants', 'indpak', 'venues', 'filipino', 'french', 'newmexican', 'seafood', 'mediterranean', 'pubs', 'delis', 'popuprestaurants', 'chicken_wings', 'turkish', 'intlgrocery', 'latin', 'asianfusion', 'sandwiches', 'tapasmallplates', 'bars', 'tex-mex', 'pancakes', 'falafel', 'tapas', 'sushi', 'soulfood', 'spanish', 'whiskeybars', 'laotian', 'thai', 'brewpubs', 'himalayan', 'sardinian', 'halal', 'british', 'bakeries', 'gourmet', 'piadina', 'poke', 'tikibars', 'burgers', 'fishnchi

Con los datos obtenidos del análisis previo y considerando la posible correlación con otras variables, se decidió imputar los valores nulos en la columna 'price' utilizando la media de los aliases (es decir, las categorías de los restaurantes). Esta decisión se basa en que otras variables, como la ubicación, el rating o las transacciones, no presentan una relación suficientemente fuerte o directa con el precio, por lo que su uso podría introducir sesgos. En cambio, el tipo de comida (representado por los aliases) es un factor determinante en el rango de precios de un restaurante, haciendo que la imputación basada en categorías sea la opción más adecuada y justificada para este caso.

Para comenzar con la imputación, primero transformamos las listas de categorías (`aliases`) de cada restaurante en una sola cadena de texto, donde las categorías aparecen separadas por comas. Esto facilita el manejo y análisis de las categorías, permitiendo agrupar y calcular estadísticas de manera más sencilla en los siguientes pasos.

In [46]:
# Crear una nueva columna con los aliases como una cadena separada por comas

df_res['aliases_2'] = df_res['aliases'].apply(lambda x: ', '.join(map(str, x))) # Columna 'aliases_2' con los aliases como una cadena separada por comas

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,4.3,"[delivery, pickup]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican"
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,4.8,"[restaurant_reservation, delivery]",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[wine_bars, seafood, steak]","wine_bars, seafood, steak"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch"
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,4.4,"[delivery, pickup]",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[mediterranean, tapasmallplates]","mediterranean, tapasmallplates"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,4.3,"[delivery, pickup]",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[korean, chicken_wings]","korean, chicken_wings"
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,4.4,"[delivery, pickup]",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[italian, desserts, cocktailbars]","italian, desserts, cocktailbars"
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[cocktailbars, burgers]","cocktailbars, burgers"
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,4.0,"[delivery, pickup]",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[korean, bbq]","korean, bbq"


Con esto, transformamos la columna de precios a un valor numérico, lo que facilita su manipulación y permite realizar operaciones estadísticas o de imputación de manera más eficiente. Por ejemplo, convertimos los símbolos de dólar (`$`, `$$`, etc.) en valores enteros:

In [47]:
# Convierte '$', '$$', '$$$', etc. a 1, 2, 3, ...

df_res['price_num'] = df_res['price'].map(lambda x: len(str(x)) if pd.notna(x) else np.nan) # Nueva columna 'price_num' con valores numéricos excluyendo nulos

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.0
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,4.3,"[delivery, pickup]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.0
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,4.8,"[restaurant_reservation, delivery]",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[wine_bars, seafood, steak]","wine_bars, seafood, steak",3.0
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.0
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,4.4,"[delivery, pickup]",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[mediterranean, tapasmallplates]","mediterranean, tapasmallplates",3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,4.3,"[delivery, pickup]",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[korean, chicken_wings]","korean, chicken_wings",2.0
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,4.4,"[delivery, pickup]",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[italian, desserts, cocktailbars]","italian, desserts, cocktailbars",3.0
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[cocktailbars, burgers]","cocktailbars, burgers",NaN
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,4.0,"[delivery, pickup]",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[korean, bbq]","korean, bbq",3.0


Teniendo esto, "aplanamos" únicamente las columnas 'aliases' y 'price_num'. Es decir, transformamos el DataFrame para que cada alias (categoría) de restaurante tenga su propio valor numérico de precio ('price_num') por fila. Esto permite analizar la relación entre las categorías de restaurantes y su rango de precios de forma más detallada y flexible.


In [48]:
# Aplana el DataFrame para que cada alias tenga el price_num de su fila original

df_alias_price = df_res[['aliases', 'price_num']].explode('aliases').reset_index(drop=True) # Explota la columna 'aliases' para que cada alias tenga su propia fila, manteniendo el 'price_num' asociado
display(df_alias_price) # Mostrar el DataFrame resultante para verificar los cambios

,aliases,price_num
0,newamerican,3.0
1,bars,3.0
2,bakeries,3.0
3,tapasmallplates,3.0
4,mediterranean,3.0
...,...,...
467,cocktailbars,NaN
468,burgers,NaN
469,korean,3.0
470,bbq,3.0


Ya teniendo una nueva 'tabla', calculamos el precio promedio (`price_num`) para cada alias (categoría de restaurante). Esto nos permite conocer el rango de precios típico asociado a cada tipo de restaurante y utilizar esta información para imputar valores faltantes de manera más precisa.

In [49]:
media_por_alias = df_alias_price.groupby('aliases')['price_num'].mean().reset_index()
display(media_por_alias)

,aliases,price_num
0,argentine,2.000000
1,asianfusion,2.200000
2,australian,NaN
3,bakeries,2.500000
4,bars,2.571429
...,...,...
96,venues,2.250000
97,vietnamese,2.000000
98,waffles,2.000000
99,whiskeybars,2.000000


Ahora creamos un diccionario que asocia cada alias con su precio promedio para poder usar la información de la manera más eficiente. Así, podremos consultar rápidamente el precio promedio de cualquier categoría.


In [50]:
# Crear un diccionario para mapear aliases a su precio promedio

media_por_alias_dict = media_por_alias.set_index('aliases')['price_num'].to_dict() # set_index convierte la columna 'aliases' en el índice del DataFrame, y to_dict convierte la Serie resultante en un diccionario donde las claves son los aliases y los valores son los precios promedio

Con esto, definimos una función para imputar los valores faltantes de 'price_num' en el DataFrame original. Para los nulos se le asigna la media de los precios de sus categorías (aliases), logrando así una imputación más informada y específica.


In [51]:
# Función para imputar usando la media de los aliases de la fila

def imputar_media_aliases(row): # row es una fila del DataFrame
    
    if pd.isna(row['price_num']): # Si el valor de 'price_num' es nulo
        
        aliases = row['aliases'] # Obtener los aliases de la fila
        
        # Si hay más de un alias, toma la media de las medias
        medias = [media_por_alias_dict[a] for a in aliases if a in media_por_alias_dict and not pd.isna(media_por_alias_dict[a])] # Lista de medias para los aliases presentes en el diccionario y que no son nulos
        
        if medias: # Si la lista de medias no está vacía
            return np.mean(medias) # Retorna la media de las medias
        
    return row['price_num'] # Si no es nulo, retorna el valor original

Ya teniendo la función que realizará la imputación, procedemos a aplicarla:

In [52]:
# Aplicar la función al DataFrame 

df_res['price_num'] = df_res.apply(imputar_media_aliases, axis=1) # Se aplica la función a cada fila del DataFrame para imputar los valores faltantes en 'price_num'

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.000000
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,4.3,"[delivery, pickup]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.000000
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,4.8,"[restaurant_reservation, delivery]",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[wine_bars, seafood, steak]","wine_bars, seafood, steak",3.000000
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.000000
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,4.4,"[delivery, pickup]",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[mediterranean, tapasmallplates]","mediterranean, tapasmallplates",3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,4.3,"[delivery, pickup]",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[korean, chicken_wings]","korean, chicken_wings",2.000000
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,4.4,"[delivery, pickup]",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[italian, desserts, cocktailbars]","italian, desserts, cocktailbars",3.000000
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[cocktailbars, burgers]","cocktailbars, burgers",2.583333
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,4.0,"[delivery, pickup]",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[korean, bbq]","korean, bbq",3.000000


Comprobamos que la función realizó la imputación correctamente, sin embargo hay valores que aún están faltantes. Esto no es un error, es debido a que falta información de esas categorías que hacen imposible que se pueda completar con la función antes declarada. Analizamos cuántos nulos quedan y si es conveniente imputar con la media global de la columna 'price_num':

In [53]:
# cuantos valores nulos quedan en price_num

df_res['price_num'].isna().sum()

np.int64(6)

Debido a que el porcentaje de nulos es bastante bajo, el imputarlos con la media global no representa un gran sesgo:


In [54]:
# imputar valores nulos con la media global de la columna price_num

df_res['price_num'].fillna(df_res['price_num'].mean(), inplace=True) # Imputar los valores nulos en 'price_num' con la media global de la columna

df_res # Mostramos el DataFrame actualizado para verificar los cambios

C:\Users\mfyz1\AppData\Local\Temp\ipykernel_23236\636028266.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_res['price_num'].fillna(df_res['price_num'].mean(), inplace=True) # Imputar los valores nulos en 'price_num' con la media global de la columna


,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.000000
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,4.3,"[delivery, pickup]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.000000
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,4.8,"[restaurant_reservation, delivery]",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[wine_bars, seafood, steak]","wine_bars, seafood, steak",3.000000
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.000000
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,4.4,"[delivery, pickup]",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[mediterranean, tapasmallplates]","mediterranean, tapasmallplates",3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,4.3,"[delivery, pickup]",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[korean, chicken_wings]","korean, chicken_wings",2.000000
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,4.4,"[delivery, pickup]",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[italian, desserts, cocktailbars]","italian, desserts, cocktailbars",3.000000
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[cocktailbars, burgers]","cocktailbars, burgers",2.583333
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,4.0,"[delivery, pickup]",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[korean, bbq]","korean, bbq",3.000000


Con ello, ya no tenemos valores nulos que puedan afectar a futuros análisis, además que se imputaron de manera inteligente. Hacemos una columna extra donde podamos identificar aquellos valores que fueron afectados y quede registro de ello:

In [55]:
# nueva columna que indique si el valor fue imputado o no

df_res['price_num_imputado'] = df_res['price'].isna() # Crear una nueva columna que indique si el valor fue imputado (True) o no (False)

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num,price_num_imputado
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.000000,False
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,4.3,"[delivery, pickup]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.000000,False
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,4.8,"[restaurant_reservation, delivery]",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[wine_bars, seafood, steak]","wine_bars, seafood, steak",3.000000,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.000000,False
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,4.4,"[delivery, pickup]",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[mediterranean, tapasmallplates]","mediterranean, tapasmallplates",3.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,4.3,"[delivery, pickup]",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[korean, chicken_wings]","korean, chicken_wings",2.000000,False
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,4.4,"[delivery, pickup]",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[italian, desserts, cocktailbars]","italian, desserts, cocktailbars",3.000000,False
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[cocktailbars, burgers]","cocktailbars, burgers",2.583333,True
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,4.0,"[delivery, pickup]",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[korean, bbq]","korean, bbq",3.000000,False


Ya completado el Data Frame, hacemos el cambio de tipo de variable en la columna 'price_num' para que solo indique números enteros

In [56]:
# Convertir a int en 'price_num' y hacer el redondeo hacia arriba si es más de 0.5

df_res['price_num'] = df_res['price_num'].round().astype(int) # Redondear los valores en 'price_num' y convertirlos a enteros

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num,price_num_imputado
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3,False
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,4.3,"[delivery, pickup]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3,False
2,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,https://www.yelp.com/biz/penumbra-chicago?adju...,996,4.8,"[restaurant_reservation, delivery]",$$$,17737722343,41.924459,-87.710898,60647,3309 W Fullerton Ave,"[wine_bars, seafood, steak]","wine_bars, seafood, steak",3,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2,False
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1858,4.4,"[delivery, pickup]",$$$,17736451400,41.886944,-87.648795,60607,"302 N Green St, Fl 3","[mediterranean, tapasmallplates]","mediterranean, tapasmallplates",3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WBU0yq9J8qiYQfI_fh2P1Q,crisp-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-cvYAt...,https://www.yelp.com/biz/crisp-chicago?adjust_...,3644,4.3,"[delivery, pickup]",$$,17736977610,41.936121,-87.644421,60657,2940 N Broadway Ave,"[korean, chicken_wings]","korean, chicken_wings",2,False
196,LYppbvgJlBG0SqjSKFiFGg,sapori-trattoria-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/AicJhq...,https://www.yelp.com/biz/sapori-trattoria-chic...,3011,4.4,"[delivery, pickup]",$$$,17738329999,41.931660,-87.648590,60614,2701 N Halsted St,"[italian, desserts, cocktailbars]","italian, desserts, cocktailbars",3,False
197,T4j4q9Z3OFTTBA39Uv6lLA,bad-habit-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/nY_1gc...,https://www.yelp.com/biz/bad-habit-chicago?adj...,16,4.3,[],NaN,17739002449,41.918715,-87.689922,60647,2047 N Milwaukee Ave,"[cocktailbars, burgers]","cocktailbars, burgers",3,True
198,DwJwTn1MMhkUk6umLeM5MQ,daebak-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/RgV3VR...,https://www.yelp.com/biz/daebak-chicago?adjust...,59,4.0,"[delivery, pickup]",$$$,13127226923,41.904660,-87.668940,60622,1266 N Milwaukee Ave,"[korean, bbq]","korean, bbq",3,False


Eliminamos la columna `price` ya que ahora contamos con la columna `price_num` completa y en formato numérico, lo que facilita futuros análisis:


In [57]:
# Elimina columna 'price'

df_res.drop(columns=['price'], inplace=True) # eliminamos la columna que ya no es necesaria 

Mostramos la información del DataFrame terminado para corroborar la limpieza y transformación exitosos:


In [58]:
df_res.info() # Verificar que no haya valores nulos y los tipos de datos sean correctos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   image_url              200 non-null    object 
 3   url                    200 non-null    object 
 4   review_count           200 non-null    int64  
 5   rating                 200 non-null    float64
 6   transactions           200 non-null    object 
 7   phone                  200 non-null    object 
 8   coordinates.latitude   200 non-null    float64
 9   coordinates.longitude  200 non-null    float64
 10  location.zip_code      200 non-null    int64  
 11  address                200 non-null    object 
 12  aliases                200 non-null    object 
 13  aliases_2              200 non-null    object 
 14  price_num              200 non-null    int64  
 15  price_

Extra: ordenar las columnas para tener las más relevantes al inicio. Esto facilita el análisis y visualización de los datos:


In [59]:
# Ordenar las columnas 

columnas_ordenadas = [
    'id', 
    'alias', 
    'aliases',
    'price_num', 
    'price_num_imputado', 
    'phone', 
    'address', 
    'rating', 
    'review_count', 
    'transactions',  
    'image_url', 
    'url', 
    'location.zip_code', 
    'coordinates.longitude', 
    'coordinates.latitude' ] # Lista con el orden deseado de las columnas

df_res = df_res[columnas_ordenadas] # Reordenar las columnas del DataFrame según la lista definida

Como último paso, exportamos este DataFrame que ya está normalizado y listo para análisis:

In [60]:
# exportar el DataFrame final a un archivo CSV

df_res.to_csv('Datos_YELP_limpios.csv', index=False) # Exportar el DataFrame limpio a un archivo CSV sin incluir el índice

## **Conclusiones generales**

----
# Índice

- [1. Conexión a la API de Yelp](#Conexion-a-la-API-de-YELP)
- [2. Análisis exploratorio de datos](#Exploracion-del-DataFrame)
- [3. Limpieza de datos](#Limpieza-de-datos)
- [4. Conclusiones](#Conclusiones-generales)
--------

### **Resumen del Trabajo Realizado**

Este segundo avance del proyecto integrador logró conectar exitosamente con la **API de Yelp** para obtener datos de restaurantes en Chicago, transformando información cruda de 200 establecimientos gastronómicos en un dataset estructurado y listo para análisis. El proceso integró la recolección de datos externos con técnicas avanzadas de limpieza, complementando la base de usuarios del Avance 1.

### **Principales Problemas Resueltos**

#### **1. Obtención y Estructuración de Datos Externos**
- **Conexión API exitosa**: Se superaron las limitaciones de la API de Yelp (50 registros máximo por consulta) mediante solicitudes múltiples con parámetros de offset
- **Normalización JSON**: Transformación eficiente de estructuras anidadas a formato tabular manejable
- **Persistencia de datos**: Almacenamiento local para evitar consultas repetitivas a la API

#### **2. Depuración de Redundancias**
- **Análisis de similitud**: Uso de FuzzyWuzzy para comparar columnas aparentemente duplicadas con precisión cuantitativa
- **Consolidación inteligente**: Unificación de direcciones fragmentadas y eliminación de teléfonos/nombres redundantes
- **Optimización estructural**: Reducción de 24 columnas iniciales a 15 variables relevantes

#### **3. Transformación de Datos Complejos**
- **Estructuras anidadas**: Conversión exitosa de strings con formato JSON a objetos Python reales usando ast.literal_eval
- **Categorización avanzada**: Extracción y normalización de categorías de restaurantes para análisis posterior
- **Estandarización numérica**: Conversión de símbolos de precio ($, $$, $$$) a valores enteros procesables

### **Metodología de Imputación Innovadora**

La estrategia de tratamiento de valores nulos fue particularmente interesante:

1. **Imputación contextual por categorías**: Los valores faltantes en precios se imputaron usando la media de restaurantes con categorías similares
2. **Respaldo estadístico**: Valores residuales se trataron con media global para minimizar sesgos
3. **Trazabilidad completa**: Creación de columna indicadora para identificar valores imputados
4. **Validación de coherencia**: Verificación de que las imputaciones mantuvieran la lógica del negocio

### **Resultados Técnicos Obtenidos**

El dataset final de restaurantes presenta:
- **Completitud**: 199 registros sin valores nulos en variables críticas
- **Consistencia**: Tipos de datos apropiados y estructura optimizada
- **Relevancia**: 15 columnas estratégicamente seleccionadas y ordenadas por importancia
- **Calidad**: Información validada y lista para análisis de mercado gastronómico

### **Integración Estratégica con Avance 1**

Este segundo entregable complementa perfectamente el trabajo previo:

- **Coherencia geográfica**: Ambos datasets están centrados en Chicago
- **Compatibilidad analítica**: Los datos de restaurantes se alinean con los perfiles de usuarios ya limpiados
- **Preparación para modelado**: Estructura lista para análisis de coincidencias y recomendaciones

### **Innovaciones Metodológicas Aplicadas**

1. **Función de imputación personalizada**: Desarrollo de lógica específica para tratamiento contextualizado de valores faltantes
2. **Análisis de similitud automatizado**: Uso de métricas cuantitativas para decisiones de consolidación
3. **Transformación de tipos de datos avanzada**: Manejo eficiente de estructuras complejas (listas, diccionarios)
4. **Optimización de estructura**: Reordenamiento estratégico de columnas por relevancia analítica

### **Valor Agregado para Análisis Posteriores**

Este trabajo proporciona:

1. **Ecosistema completo**: Datos tanto de usuarios como de establecimientos gastronómicos
2. **Base para recomendaciones**: Información estructurada para modelos de matching
3. **Insights de mercado**: Datos listos para análisis de tendencias gastronómicas en Chicago
4. **Escalabilidad**: Metodología replicable para otras ciudades

### **Próximos Pasos del Proyecto**

Los siguientes avances incluirán:
- **Análisis exploratorio conjunto** de usuarios y restaurantes
- **Desarrollo de modelos de recomendación** basados en preferencias alimenticias y características de establecimientos
- **Identificación de oportunidades de mercado** mediante cruce de datos de oferta y demanda
- **Generación de insights accionables** para estrategias de marketing personalizado

### **Desafíos Superados**

1. **Limitaciones de API**: Maximización de datos obtenidos dentro de restricciones técnicas
2. **Complejidad estructural**: Manejo exitoso de datos anidados y formatos diversos
3. **Calidad heterogénea**: Estrategias diferenciadas según el tipo y contexto de cada variable
4. **Coherencia temática**: Mantenimiento de relevancia para el contexto gastronómico de Chicago

### **Lecciones Aprendidas**

Este proceso destacó la importancia de:
- **Planificación de APIs**: Entender limitaciones antes de implementar soluciones
- **Flexibilidad metodológica**: Adaptar técnicas de limpieza según las características específicas de cada dataset
- **Validación continua**: Verificar la coherencia de transformaciones en cada paso
- **Documentación exhaustiva**: Registrar decisiones para asegurar reproducibilidad y comprensión

### **Reflexión Final**

La obtención y limpieza de datos de restaurantes no solo completó el ecosistema de información necesario para el proyecto, sino que demostró la importancia de integrar fuentes de datos externas de manera sistemática y rigurosa. El dataset de restaurantes de Chicago representa ahora un complemento estratégico que, junto con los datos de usuarios del Avance 1, proporciona una visión 360° del mercado gastronómico.

Este segundo avance establece las bases técnicas y metodológicas para análisis avanzados que permitirán a InsightReach desarrollar estrategias de marketing digital altamente personalizadas y efectivas.

---

> **Nota**: Este segundo avance completa la fase de preparación de datos del proyecto integrador. Los análisis específicos, modelos predictivos y recomendaciones estratégicas se desarrollarán en los siguientes entregables, aprovechando la solidez y calidad de ambos datasets integrados.